<h1 align='center'>Введение в обработку естественного языка</h1>  
<h2 align='center'>Урок 13. Модель BERT и GPT</h2>  
Практическое задание:  

Взять датасет https://huggingface.co/datasets/merionum/ru_paraphraser решить задачу парафраза

(дополнительно необязательная задача)на выбор взять https://huggingface.co/datasets/sberquad https://huggingface.co/datasets/blinoff/medical_qa_ru_data натренировать любую модель для вопросно ответной системы как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER




Загрузим датасет и необходимые библиотеки

In [ ]:
!pip install -qq datasets

In [ ]:
# !pip install -q transformers==4.28.0

In [ ]:
!pip install -q git+https://github.com/huggingface/accelerate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import numpy as np
import pandas as pd

import torch
from torch.utils.data import DataLoader, Dataset

from datasets import load_dataset

from transformers import GPT2LMHeadModel, GPT2Tokenizer, DataCollatorForLanguageModeling, TextDataset, TrainingArguments, Trainer

## Загрузим данные с помощью библиотеки datasets

In [ ]:
dataset = load_dataset('merionum/ru_paraphraser')

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset Summary  
ParaPhraser — это корпус заголовков новостей, аннотированный по следующей схеме:

1: precise paraphrases (точныq пересказ)  
0: near paraphrases (близкий пересказ)  
-1: non-paraphrases (не перезсказ)  

In [ ]:
train = dataset['train']
test = dataset['test']
train

Dataset({
    features: ['id', 'id_1', 'id_2', 'text_1', 'text_2', 'class'],
    num_rows: 7227
})

In [ ]:
train[10]

{'id': '11',
 'id_1': '246',
 'id_2': '8165',
 'text_1': 'Москвичи смогут забронировать в Интернете место на кладбище.',
 'text_2': 'В Москве можно будет забронировать место на кладбище через интернет.',
 'class': '1'}

Берём только точные paraphrases

In [ ]:
threshold = 0
train_x = np.array(train['text_1'])[np.array(train['class']).astype(int)>threshold]
train_y = np.array(train['text_2'])[np.array(train['class']).astype(int)>threshold]
test_x = np.array(test['text_1'])[np.array(test['class']).astype(int)>threshold]
test_y = np.array(test['text_2'])[np.array(test['class']).astype(int)>threshold]
train_x[0], train_y[0]

('Приставы соберут отпечатки пальцев российских должников.',
 'Приставы снимут отпечатки пальцев у злостных неплательщиков.')

Так как оценка будет визуальной, то для теста возьмём только 15 примеров

In [ ]:
df = pd.DataFrame({'original_text': list(train_x)+list(test_x)[:15], 'paraphrased_text': list(train_y)+list(test_y)[:15]})
df.head()

,original_text,paraphrased_text
0,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...
1,Москвичи смогут забронировать в Интернете мест...,В Москве можно будет забронировать место на кл...
2,Северокорейский лидер впервые за 19 лет поздра...,Лидер КНДР впервые за 19 лет поздравил согражд...
3,Мужчина из Подмосковья случайно убил жену в Но...,Житель Подмосковья случайно убил жену на новог...
4,Житель Украины расстрелял посетителей кафе.,На Украине мужчина через окно расстрелял посет...


In [ ]:
test=list(zip(list(test_x)[-15:], list(test_y)[-15:]))
test[0]

('США исключили Кубу из списка государств, спонсирующих терроризм',
 'США исключили Кубу из списка государств-спонсоров терроризма')

Добавим спец символы, обозначающие начало и конецфраз, а так же их разделение

In [ ]:
df['combined'] = '<s>' + df['original_text'] + '<s>'+' --> '+'<p>'+df['paraphrased_text']+'</p>'
df.head()

,original_text,paraphrased_text,combined
0,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...,<s>Приставы соберут отпечатки пальцев российск...
1,Москвичи смогут забронировать в Интернете мест...,В Москве можно будет забронировать место на кл...,<s>Москвичи смогут забронировать в Интернете м...
2,Северокорейский лидер впервые за 19 лет поздра...,Лидер КНДР впервые за 19 лет поздравил согражд...,<s>Северокорейский лидер впервые за 19 лет поз...
3,Мужчина из Подмосковья случайно убил жену в Но...,Житель Подмосковья случайно убил жену на новог...,<s>Мужчина из Подмосковья случайно убил жену в...
4,Житель Украины расстрелял посетителей кафе.,На Украине мужчина через окно расстрелял посет...,<s>Житель Украины расстрелял посетителей кафе....


In [ ]:
df['combined'][0]

'<s>Приставы соберут отпечатки пальцев российских должников.<s> --> <p>Приставы снимут отпечатки пальцев у злостных неплательщиков.</p>'

Сохраним в текстовый файл наши данные

In [ ]:
df.combined.to_csv('combined.txt', sep='\n', index=False)

### Модель

Используем gpt2 модель, которую проходили на вебинаре

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2", eos_token='</p>')
ru_gpt = GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2", output_attentions=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
device = "cuda:0"
ru_gpt.to(device);

Подготовка к тренировке модели

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [ ]:
train_path='/content/combined.txt'
train_dataset=TextDataset(tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)

/usr/local/lib/python3.10/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Настройка параметров обучения

In [ ]:
training_args = TrainingArguments(
    output_dir="./gpt2-chief", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=4, # batch size for training
    per_device_eval_batch_size=4,  # batch size for evaluation
    eval_steps = 100, # Number of update steps between two evaluations.
    save_steps=100, # after # steps model is saved
    warmup_steps=100,# number of warmup steps for learning rate scheduler
    logging_steps=100
    )

In [ ]:
trainer = Trainer(
    model=ru_gpt,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

Но сначала посмотрим, что модель генерирует без обучения

In [ ]:
# def paraphrase(text, repetition_penalty=5., temperature=0.5, num_beams=10):
def paraphrase(text, repetition_penalty, temperature, num_beams):    
    ru_gpt.eval()
    spec_text = '<s>'+text+'</s>'+' --> '+'<p>'
    tokens = tokenizer(spec_text, return_tensors='pt').to('cuda:0')
    size = tokens['input_ids'].shape[1]
    output = ru_gpt.generate(
        **tokens,
        do_sample=False,
        max_length=size+50, 
        repetition_penalty=repetition_penalty, 
        temperature=temperature,
        num_beams=num_beams,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(output[0])
    result = decoded[len(spec_text):]
    stop_indx = result.index('</p>') if '</p>' in result else None
    result = result[:stop_indx]
    return result

In [ ]:
for text in test[:3]:
  print(f'Input: {text[0]}')
  print(f'Paraphrase: {paraphrase(text[0], repetition_penalty=5., temperature=0.5, num_beams=10)}')
  print(f'True: {text[1]}')
  print()

Input: США исключили Кубу из списка государств, спонсирующих терроризм
Paraphrase: 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&
True: США исключили Кубу из списка государств-спонсоров терроризма

Input: СМИ: налоговая служба США готовит новые обвинения по делу ФИФА
Paraphrase: 
http://www.gazeta.ru/business/news/2015/08/31/n_14623949.shtml


31382471	makowkina-nata
True: Налоговая служба США пообещала новые обвинения по делу о коррупции в ФИФА

Input: Около 20 стран будут участвовать в армейских играх в России в августе
Paraphrase: http://www.youtube.com/...✂ http://www.youtube.com/...✂ http://www.youtube.com/...✂ http://www.youtube.com/...✂ http://www.youtube.com/...✂ http://www.y
True: В армейских играх в России примут участие около 20 стран



Как видно, модель совсем необучена на задачу Paraphrase.  
Займёмся её тренеровкой

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
100,2.384100
200,1.569000
300,1.346600


TrainOutput(global_step=378, training_loss=1.6469452873108879, metrics={'train_runtime': 83.1385, 'train_samples_per_second': 18.187, 'train_steps_per_second': 4.547, 'total_flos': 98768388096000.0, 'train_loss': 1.6469452873108879, 'epoch': 3.0})

Посмотрим какие теперь результаты:

In [ ]:
for text in test[:3]:
  print(f'Input: {text[0]}')
  print(f'Paraphrase: {paraphrase(text[0], repetition_penalty=5., temperature=0.5, num_beams=10)}')
  print(f'True: {text[1]}')
  print()

Input: США исключили Кубу из списка государств, спонсирующих терроризм
Paraphrase: США исключили Кубу из списка стран, запрещенных в России 
True: США исключили Кубу из списка государств-спонсоров терроризма

Input: СМИ: налоговая служба США готовит новые обвинения по делу ФИФА
Paraphrase: Американская налоговая служба заявила о новых обвинениях в адрес ФИФА 
True: Налоговая служба США пообещала новые обвинения по делу о коррупции в ФИФА

Input: Около 20 стран будут участвовать в армейских играх в России в августе
Paraphrase: КНДР объявила о проведении военных игр в России в августе 
True: В армейских играх в России примут участие около 20 стран



Результаты очень даже неплохие

Попробуем улучшить генерацию увеличивая параметр beem

In [ ]:
for text in test[:3]:
  print(f'Input: {text[0]}')
  print(f'Paraphrase: {paraphrase(text[0], repetition_penalty=5., temperature=0.5, num_beams=150)}')
  print(f'True: {text[1]}')
  print()

Input: США исключили Кубу из списка государств, спонсирующих терроризм
Paraphrase: США исключили Кубу из списка стран, запрещенных в России 
True: США исключили Кубу из списка государств-спонсоров терроризма

Input: СМИ: налоговая служба США готовит новые обвинения по делу ФИФА
Paraphrase: Биржевой курс доллара упал ниже 50 руб. 
True: Налоговая служба США пообещала новые обвинения по делу о коррупции в ФИФА

Input: Около 20 стран будут участвовать в армейских играх в России в августе
Paraphrase: Биржевой курс доллара упал ниже 50 руб. 
True: В армейских играх в России примут участие около 20 стран



Теперь попробуем уменьшить температуру

In [ ]:
for text in test[:3]:
  print(f'Input: {text[0]}')
  print(f'Paraphrase: {paraphrase(text[0], repetition_penalty=5., temperature=0.1, num_beams=10)}')
  print(f'True: {text[1]}')
  print()

Input: США исключили Кубу из списка государств, спонсирующих терроризм
Paraphrase: США исключили Кубу из списка стран, запрещенных в России 
True: США исключили Кубу из списка государств-спонсоров терроризма

Input: СМИ: налоговая служба США готовит новые обвинения по делу ФИФА
Paraphrase: Американская налоговая служба заявила о новых обвинениях в адрес ФИФА 
True: Налоговая служба США пообещала новые обвинения по делу о коррупции в ФИФА

Input: Около 20 стран будут участвовать в армейских играх в России в августе
Paraphrase: КНДР объявила о проведении военных игр в России в августе 
True: В армейских играх в России примут участие около 20 стран



In [ ]:
for text in test[:3]:
  print(f'Input: {text[0]}')
  print(f'Paraphrase: {paraphrase(text[0], repetition_penalty=5., temperature=1, num_beams=10)}')
  print(f'True: {text[1]}')
  print()

Input: США исключили Кубу из списка государств, спонсирующих терроризм
Paraphrase: США исключили Кубу из списка стран, запрещенных в России 
True: США исключили Кубу из списка государств-спонсоров терроризма

Input: СМИ: налоговая служба США готовит новые обвинения по делу ФИФА
Paraphrase: Американская налоговая служба заявила о новых обвинениях в адрес ФИФА 
True: Налоговая служба США пообещала новые обвинения по делу о коррупции в ФИФА

Input: Около 20 стран будут участвовать в армейских играх в России в августе
Paraphrase: КНДР объявила о проведении военных игр в России в августе 
True: В армейских играх в России примут участие около 20 стран



По-моему, стало хуже

На самом деле gpt состоит только из декодера, что для данной задачи не очень хорошо, так как мы хотим не придумывать совсем что то новое, а больше ориентироваться на входные данные. 

Поэтому в этой задаче лучше подойдёт такая архитектура как Т5. В ней есть и энкодер и декодер.

In [ ]:
!pip install -q transformers[SentencePiece]

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [ ]:
tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-base-paraphraser")
ru_t5 = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-base-paraphraser", output_attentions=True)

In [ ]:
ru_t5.to('cuda:0');

In [ ]:
def paraphrase2(text, repetition_penalty, temperature, num_beams):
  ru_t5.eval()
  spec_text = '<s>'+text
  tokens = tokenizer(spec_text, return_tensors='pt').to('cuda:0')
  size = tokens['input_ids'].shape[1]
  output = ru_t5.generate(
    **tokens,
    do_sample=False,
    max_length=size+50, 
    repetition_penalty=repetition_penalty, 
    temperature=temperature,
    num_beams=num_beams
  )

  result = tokenizer.decode(output[0])
  pad_indx = result.index('<pad>')+5 if '<pad>' in result else None
  stop_indx = result.index('</s>') if '</s>' in result else None
  result = result[pad_indx:stop_indx]
  return result

In [ ]:
for text in test:
  print(f'Input: {text[0]}')
  print(f'Paraphrase: {paraphrase2(text[0], repetition_penalty=5., temperature=0.5, num_beams=10)}')
  print(f'True: {text[1]}')
  print()

Input: США исключили Кубу из списка государств, спонсирующих терроризм
Paraphrase:  США исключили Кубу из списка государств, спонсирующих терроризм
True: США исключили Кубу из списка государств-спонсоров терроризма

Input: СМИ: налоговая служба США готовит новые обвинения по делу ФИФА
Paraphrase:  В МИД РФ заявили, что налоговая служба США готова принять новые обвинения по делу ФИФА
True: Налоговая служба США пообещала новые обвинения по делу о коррупции в ФИФА

Input: Около 20 стран будут участвовать в армейских играх в России в августе
Paraphrase:  <s>Около 20 стран будут участвовать в армейских играх в России
True: В армейских играх в России примут участие около 20 стран

Input: Порошенко назначил Саакашвили губернатором Одесской области
Paraphrase:  Порошенко назначил Саакашвили губернатором Одесской области
True: Порошенко объявил о назначении Саакашвили одесским губернатором

Input: Порошенко назначил Саакашвили главой Одесской области
Paraphrase:  Порошенко назначил Саакашвили г

Как видно, это модель намного точнее, но при этом менее вариативнее.